In [1]:
from datasets import load_from_disk
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments
from datasets import load_dataset, Audio
import os
from scipy.io import wavfile
import numpy as np
import torch
from pydub import AudioSegment

In [2]:
ds_arrays = load_from_disk("/kaggle/input/mathbridge-audio-arrays/mathbridge_audio_arrays")
ds_arrays_train = ds_arrays.select(range(80000))
ds_arrays_test = ds_arrays.select(range(80000, 99986))
ds_arrays_train, ds_arrays_test

Loading dataset from disk:   0%|          | 0/33 [00:00<?, ?it/s]

(Dataset({
     features: ['audio'],
     num_rows: 80000
 }),
 Dataset({
     features: ['audio'],
     num_rows: 19986
 }))

In [3]:
ds_arrays_train[0], ds_arrays_test[0]

({'audio': {'path': 'tts_0.mp3',
   'array': array([ 0.03302002,  0.0010376 , -0.02728271, ...,  0.        ,
           0.        ,  0.        ]),
   'sampling_rate': 22050}},
 {'audio': {'path': 'tts_8201.mp3',
   'array': array([ 0.01837158,  0.00418091, -0.00866699, ...,  0.        ,
           0.        ,  0.        ]),
   'sampling_rate': 22050}})

In [4]:
ds = load_from_disk("/kaggle/input/mathbridge-filtered/mathbridge_filtered").select(range(100000))
ds

Dataset({
    features: ['context_before', 'equation', 'context_after', 'spoken_English'],
    num_rows: 100000
})

In [5]:
ds[0]

{'context_before': 'are modeled by a stochastic control process with variance',
 'equation': '$ \\sigma^2_t $',
 'context_after': 'controlled by the agent and with a mean of zero . This models potential effect of actions centered around the null action . To compute various quantities of interest ,',
 'spoken_English': 'sigma squared sub t.'}

In [6]:
# Load pre-trained model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
ds_arrays_train = ds_arrays_train.cast_column("audio", Audio(sampling_rate=16_000))
ds_arrays_test = ds_arrays_test.cast_column("audio", Audio(sampling_rate=16_000))

In [8]:
ds_arrays_train[0], ds_arrays_test[0]

({'audio': {'path': 'tts_0.mp3',
   'array': array([ 0.02826029, -0.01515469, -0.00925331, ...,  0.        ,
           0.        ,  0.        ]),
   'sampling_rate': 16000}},
 {'audio': {'path': 'tts_8201.mp3',
   'array': array([ 0.01642108, -0.0040214 , -0.00188781, ...,  0.        ,
           0.        ,  0.        ]),
   'sampling_rate': 16000}})

In [9]:
def prepare_dataset(batch):
    audio = batch["audio"]
    path = audio["path"]
    id = int(path.split("_")[-1].split(".")[0])
    batch["input_features"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_features"])
    batch["labels"] = processor(text=ds[id]["equation"]).input_ids
    batch["path"] = path
    return batch

In [10]:
ds_arrays_train = ds_arrays_train.select(range(10000)).map(prepare_dataset, remove_columns=ds_arrays_train.column_names, keep_in_memory=True)
ds_arrays_test = ds_arrays_test.select(range(1000)).map(prepare_dataset, remove_columns=ds_arrays_test.column_names, keep_in_memory=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
ds_arrays_train.save_to_disk("train_data")
ds_arrays_test.save_to_disk("test_data")

Saving the dataset (0/6 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]